In [ ]:
dataset = pd.read_csv("/kaggle/input/combineddata/combinedBalancedDataset.csv", header = None, delimiter = ',')

X = dataset.drop(columns = [26,27,28,29])
y = dataset.filter([26,27,28,29])
print(X,y)

unique_rows = np.unique(y, axis=0).tolist()
print(unique_rows)

y['combined']= y.values.tolist()
y = y.drop(columns = [26,27,28,29])
print(y)
print(y.shape)
#merges mutliple targets into single multiclassfication for y
i = 0
while i < len(y):
    j = 0
    while j < len(unique_rows):
        if y.loc[y.index[i],'combined'] == unique_rows[j]:
            y.loc[y.index[i],'combined'] = unique_rows.index(unique_rows[j])
        j +=1
    i+=1
    
y['combined'] = y['combined'].astype(int)

y.to_numpy()

print(y)


# Split dataset into training and validation sets
x_train2, x_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.2, random_state=42)

print("done")
print(x_train.shape)
print(y_train.shape)

# Convert y to 1-dimensional array
y_labels = np.argmax(y.values, axis=1)


# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

x_train2, x_test2, y_train2, y_test2 = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

max_score_SMOTE = 0
best_params_SMOTE = {}

# Perform grid search for best parameters for the SMOTE dataset
for n_estimators in param_grid['n_estimators']:
    for max_depth in param_grid['max_depth']:
        for min_samples_split in param_grid['min_samples_split']:
            for min_samples_leaf in param_grid['min_samples_leaf']:
                for max_features in param_grid['max_features']:
                    params = {
                        'n_estimators': n_estimators,
                        'max_depth': max_depth,
                        'min_samples_split': min_samples_split,
                        'min_samples_leaf': min_samples_leaf,
                        'max_features': max_features
                    }
                    forest_SMOTE = RandomForestClassifier(**params)
                    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
                    scores2 = cross_val_score(forest_SMOTE, x_train2, y_train2, cv=skf)
                    score_mean2 = scores2.mean()
                    if score_mean2 > max_score_SMOTE:
                        max_score_SMOTE = score_mean2
                        best_params_SMOTE = params

print("Best Parameters for SMOTE dataset:")
print(best_params_SMOTE)
print("Average Accuracy:", max_score_SMOTE)
f1_score_final2 = 0.0
prc_auc_final2 = 0.0
roc_auc_final2 = 0.0
y_test_all2 = []
y_pred_proba_all2 = []

for train_index, test_index in skf.split(x_train2, y_train2):
    x_train_fold2, x_val_fold2 = x_train2[train_index], x_train2[test_index]
    y_train_fold2, y_val_fold2 = y_train2[train_index], y_train2[test_index]
    
    y_train_fold2 = np.ravel(y_train_fold2)
    
    final_forest_SMOTE = RandomForestClassifier(**best_params_SMOTE)
    final_forest_SMOTE.fit(x_train_fold2, y_train_fold2)
    y_pred_final_SMOTE = final_forest_SMOTE.predict_proba(x_val_fold2)[:, 1]  # Predict probabilities
    
    y_test_all2.extend(y_val_fold2)
    y_pred_proba_all2.extend(y_pred_final_SMOTE)
    
    f1_final_SMOTE = f1_score(y_val_fold2, (y_pred_final_SMOTE > 0.5).astype(int))
    f1_score_final2 += f1_final_SMOTE
    
    precision2, recall2, _ = precision_recall_curve(y_val_fold2.ravel(), y_pred_final_SMOTE.ravel())
    prc_auc2 = average_precision_score(y_val_fold2.ravel(), y_pred_final_SMOTE.ravel())
    prc_auc_final2 += prc_auc2
    
fpr_final2, tpr_final2, _ = roc_curve(y_test_all2, y_pred_proba_all2)
roc_auc_final2 = roc_auc_score(y_test_all2, y_pred_proba_all2)

f1_score_final2 /= skf.n_splits
prc_auc_final2 /= skf.n_splits

conf_matrix_final2 = confusion_matrix(y_test_all2, (np.array(y_pred_proba_all2) > 0.5).astype(int))

tpr_final_value2 = conf_matrix_final2[1, 1] / (conf_matrix_final2[1, 1] + conf_matrix_final2[1, 0])
fpr_final_value2 = conf_matrix_final2[0, 1] / (conf_matrix_final2[0, 1] + conf_matrix_final2[0, 0])

print("F1 Score for SMOTE dataset:", f1_score_final2)
print("Precision for SMOTE dataset: ", precision2)
print("Recall for SMOTE dataset:", recall2)
print("Precision-Recall AUC for SMOTE dataset:", prc_auc_final2)
print("ROC AUC for SMOTE dataset:", roc_auc_final2)
print("TPR for SMOTE roc:", tpr_final2)
print("FPR for SMOTE roc:", fpr_final2)
print("TPR for SMOTE:", tpr_final_value2)
print("FPR for SMOTE:", fpr_final_value2)

plt.figure()
plt.plot(fpr_final2, tpr_final2, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc_final2)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve for SMOTE')
plt.legend(loc="lower right")
plt.savefig('/kaggle/working/roc_curve_smote.png')
plt.show()

plt.figure()
plt.plot(recall2, precision2, color='blue', lw=2, label='Precision-Recall curve (AUC = %0.2f)' % prc_auc_final2)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for SMOTE')
plt.legend(loc="lower left")
plt.savefig('/kaggle/working/precision_recall_curve_smote.png')
plt.show()

In [ ]:
import json   
data = {
    "Average F1 Score": f1_score_final2, 
    "Precision": precision2.tolist(), 
    "Recall": recall2.tolist(), 
    "Average True Positive Rate": tpr_final_value2, 
    "Average False Positive Rate": fpr_final_value2, 
    "Average ROC AUC": roc_auc_final2, 
    "Average PRC AUC": prc_auc_final2,
    "TPR Array" : tpr_final2.tolist(),
    "FPR Array" : fpr_final2.tolist(),
}

# Specify the file path
file_path = "/kaggle/working/Random Forest - Synthetic Dataset - SMOTE.json"

# Write the data to a JSON file
with open(file_path, "w") as json_file:
    json.dump(data, json_file)